# Stable Diffusion img2img Pipeline

## Install libraries 

In [1]:
!pip install -q transformers diffusers accelerate torch==1.13.1

In [2]:
!pip install -q "ipywidgets>=7,<8" ftfy

## Authenticate with the Hugging Face Hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Import dependencies

In [29]:
from diffusers import StableDiffusionImg2ImgPipeline, EulerDiscreteScheduler
from pathlib import Path
from PIL import Image
import torch
import re

## Remove non-word characters and foreign characters

In [30]:
def slugify(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", "-", text)
    return text

## Define model

In [ ]:
model_id = "stabilityai/stable-diffusion-2"

## Pull down dataset from Roboflow
### Add in your API key below

In [50]:
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY_HERE")
project = rf.workspace("roboflow-universe-projects").project("construction-site-safety")
dataset = project.version(1).download("coco")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Construction-Site-Safety-1 in coco:: 100%|██████████| 406/406 [00:00<00:00, 6045.21it/s]


## Create a list of images and put them in proper format/size
#### Replace images below with your image locations

In [51]:
images = ["/home/ec2-user/SageMaker/Construction-Site-Safety-1/train/000246_jpg.rf.803c6bf16e1d86b997796ebb8b4b2152.jpg",
              "/home/ec2-user/SageMaker/Construction-Site-Safety-1/train/000830_jpg.rf.a21dcfb4aa17f2b4c0d22ba91549b7db.jpg",
              "/home/ec2-user/SageMaker/Construction-Site-Safety-1/train/004779_jpg.rf.92c537eed971d0111cd63ddf4589d77b.jpg"]

In [52]:
init_images = [Image.open(image).convert("RGB").resize((768,768)) for image in images]

## Define prompts with negative prompts

In [53]:
prompts = ["construction worker in snowy landscape",
           "construction worker in dark evening", 
           "construction worker in rain storm"]

negative_prompts = ["blurry, dark photo, blue",
                    "blurry, dark photo, blue",
                    "blurry, dark photo, blue"]

## Create scheduler and pipeline

In [42]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Use the Euler scheduler here instead of default
scheduler = EulerDiscreteScheduler.from_pretrained(
    model_id, subfolder="scheduler")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to(device)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.


## Create dir for storing generated images

In [43]:
DIR_NAME="./images/"
dirpath = Path(DIR_NAME)
# create parent dir if doesn't exist
dirpath.mkdir(parents=True, exist_ok=True)

## Define pipeline parameters and generate images based on prompts

In [54]:
steps = 20
scale = 9
num_images_per_prompt = 1
seed = torch.randint(0, 1000000, (1,)).item()
generator = torch.Generator(device=device).manual_seed(seed)
output = pipe(prompts, negative_prompt=negative_prompts, image=init_images, num_inference_steps=steps,
             guidance_scale=scale, num_images_per_prompt=num_images_per_prompt, generator=generator)

  0%|          | 0/16 [00:00<?, ?it/s]

## iterate through and push generated images to images dir

In [55]:
for idx, (image,prompt) in enumerate(zip(output.images, prompts*num_images_per_prompt)):
    image_name = f'{slugify(prompt)}-{idx}.png'
    image_path = dirpath / image_name
    image.save(image_path)

## Upload Images to Roboflow

In [56]:
import os
HOME = os.getcwd()
image_dir = os.path.join(HOME, "images", "")

In [57]:
image_dir

'/home/ec2-user/SageMaker/images/'

### Add in your API key and project name

In [58]:
import glob

## DEFINITIONS
# glob params
file_extension_type = ".png"

## INIT
# roboflow pip params
rf = Roboflow(api_key="YOUR_API_KEY")
upload_project = rf.workspace().project("YOUR_PROJECT")

## MAIN
# glob images
image_glob = glob.glob(image_dir + '/*' + file_extension_type)

# perform upload
for image in image_glob:
    upload_project.upload(image, num_retry_uploads=3)
    print("*** Processing image [" + str(len(image_glob)) + "] - " + image + " ***")

loading Roboflow workspace...
loading Roboflow project...
*** Processing image [6] - /home/ec2-user/SageMaker/images/construction-worker-in-dark-evening-1.png ***
*** Processing image [6] - /home/ec2-user/SageMaker/images/construction-worker-in-snowy-landscape-0.png ***
*** Processing image [6] - /home/ec2-user/SageMaker/images/blue-eyes-and-a-ponytail-2.png ***
*** Processing image [6] - /home/ec2-user/SageMaker/images/sunglasses-and-a-warm-hat-0.png ***
*** Processing image [6] - /home/ec2-user/SageMaker/images/construction-worker-in-rain-storm-2.png ***
*** Processing image [6] - /home/ec2-user/SageMaker/images/eye-glasses-and-a-baseball-cap-1.png ***
